In [1]:
## Preamble: Package Loading
import numpy as np
import ipywidgets as ipw
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import json
import kernel as kr
import psc_sumdisp as psd 
# Preamble working directory retreival

<h2>Trial Set 1: Varying Number of Time Periods </h2> 

<h3> Trial Set 1: Data Loading and Organization </h3> 

In [10]:
inpt_filenames0 = ['pscout_6_12_1954.json' ,'pscout_6_12_1220.json' , 'pscout_6_12_1799.json']
line_nms0 = ['n=30', 'n=50' ,'n=70']

res_out0 = [psd.psc_load(inpt_filenames0[i]) for i in range(len(inpt_filenames0))]
estin_dcts0 = [res_out0[i][0] for i in range(len(inpt_filenames0))]
dgp_sum_filenames0 = [ estin_dcts0[i]['input_filename'].replace('pscdata','pscsum')
                      for i in range(len(inpt_filenames0))]
dgp_dicts0 = [psd.pscsum_load(dgp_sum_filenames0[i]) 
             for i in range(len(dgp_sum_filenames0))]
dgpin_dcts0 =  [dgp_dicts0[i][0] for i in range(len(inpt_filenames0))]
merged_dcts0 = [{**estin_dcts0[i],**dgpin_dcts0[i]} for i in range(len(inpt_filenames0))]
true_bcoeffs0 = [dgp_dicts0[i][1] for i in range(len(inpt_filenames0))]
true_acoeffs0 = [dgp_dicts0[i][2] for i in range(len(inpt_filenames0))]
bcoeff0  = [res_out0[i][1] for i in range(len(inpt_filenames0))]
acoeff0  = [res_out0[i][3] for i in range(len(inpt_filenames0))]
btables0 = [res_out0[i][2] for i in range(len(inpt_filenames0))]
atables0 = [res_out0[i][4] for i in range(len(inpt_filenames0))]

<h3> Trial Set 1: Merged DGP and Estimator Function Input Dictionary Comparison </h3> 

In [11]:
psd.indict_dsp(merged_dcts0,1)

<h3> Trial Set 1: True Primary Equations Coefficients Comparison </h3>

In [5]:
psd.indict_dsp(true_bcoeffs0,1)

<h3> Trial Set 1: True Secondary Equation Coefficients Comparison </h3> 

In [6]:
psd.indict_dsp(true_acoeffs0,2)

<h3> Trial Set 1: Primary Function Coefficient Estimates </h3>

In [7]:
display(psd.cfs_dsp(bcoeff0,btables0,1,12,line_nms0))

<h3> Trial Set 1: Secondary Function Coefficient Estimates </h3>

In [8]:
display(psd.cfs_dsp(acoeff0,atables0,2,5,line_nms0))

<h2> Trial Set 2: Varying the number of crossections </h2>

<h3> Trial Set 2: Data Loading and Organization </h3> 

In [12]:
inpt_filenames1 = ['pscout_6_12_1220.json' ,'pscout_6_13_1914.json'
                   ,'pscout_6_13_1498.json','pscout_6_13_1227.json' ]
line_nms1 = ['ncr = 5','ncr = 10', 'ncr = 15', 'ncr = 20']

res_out1 = [psd.psc_load(inpt_filenames1[i]) for i in range(len(inpt_filenames1))]
estin_dcts1 = [res_out1[i][0] for i in range(len(inpt_filenames1))]
dgp_sum_filenames1 = [ estin_dcts1[i]['input_filename'].replace('pscdata','pscsum')
                      for i in range(len(inpt_filenames1))]
dgp_dicts1 = [psd.pscsum_load(dgp_sum_filenames1[i]) 
             for i in range(len(dgp_sum_filenames1))]
merged_dcts1 = [{**estin_dcts0[i],**dgpin_dcts0[i]} for i in range(len(inpt_filenames0))]
dgpin_dcts1 =  [dgp_dicts1[i][0] for i in range(len(inpt_filenames1))]
true_bcoeffs1 = [dgp_dicts1[i][1] for i in range(len(inpt_filenames1))]
true_acoeffs1 = [dgp_dicts1[i][2] for i in range(len(inpt_filenames1))]
bcoeff1  = [res_out1[i][1] for i in range(len(inpt_filenames1))]
acoeff1  = [res_out1[i][3] for i in range(len(inpt_filenames1))]
btables1 = [res_out1[i][2] for i in range(len(inpt_filenames1))]
atables1 = [res_out1[i][4] for i in range(len(inpt_filenames1))]

<h3> Trial Set 2: DGP and Estimator Function Input Dictionary Comparison </h3> 

In [13]:
psd.indict_dsp(merged_dcts1,1)

<h3> Trial Set 2: True Primary Equations Coefficients Comparison </h3>

In [14]:
psd.indict_dsp(true_bcoeffs1,1)

<h3> Trial Set 2: True Secondary Equation Coefficients Comparison </h3> 

In [15]:
psd.indict_dsp(true_acoeffs1,2)

<h3> Trial Set 2: Primary Function Coefficient Estimates </h3>

In [16]:
display(psd.cfs_dsp(bcoeff1,btables1,1,12,line_nms1))

<h3> Trial Set 2: Secondary Function Coefficient Estimates </h3>

In [17]:
display(psd.cfs_dsp(acoeff1,atables1,2,8,line_nms1))